# Рекуррентные нейронные сети (RNN)

Существует особая архитектура сетей для обработки последовательных данных. Иными словами такие данные, последовательность которых имеет значение. Это могут быть временные ряды, обработка предложений на естественном языке, звукозапись речи или обрабатывать сцены видеопотока. Главным образом, RNN на сегодняшний день применяются в работе с языковыми моделями, в частности в анализе контекста и общей связи слов в тексте и в системах машинного перевода. Такое возможно посольку рекуррентные сети, в отличие от многослойных перцептронов, могут использовать свою внутреннюю память для обработки последовательностей произвольной длины и улавливать контекст.

Под *внутренней памятью* в рекурентных нейронных сетях подразумевается наличие специального вектора - *вектора внутреннего состояния*. В этом векторе может храниться как кратковременная, контекстуальная информация, так и долговременная, более общая информация. 

При вычислении ответа при прямом проходе рекурентного слоя учитывается не только вектор признаков $X$, но и внутреннее состояние $C$, посчитанное на предыдущем проходе. Таким образом получается, что одному и тому же вектору признаков $X$, вообще говоря, могут соответвовать разные ответы, что невозможно для сетей, построенных на полносвязных и свёрточных слоях.

Схематически рекуррентный слой можно изобразить следующим образом:

<center><img src='img/RNNshort.png' width=100></center>

Рекуррентную сеть можно рассматривать, как несколько копий одной и той же сети, каждая из которых передает информацию последующей копии. Вот, что произойдет, если мы развернем обратную связь:

<center><img src='img/RNN.png' width=600></center>

Индекс у $X$ и $h$ соответствует порядковому вектора признаков и соответствующего ему выходного вектора. В случае обработки временного ряда, можно интерпретировать его как момент времени.

## Long Short-term Memory (LSTM)

В основе большинства современных RNN лежит архитектура LSTM, разработанная аж в 1997 году. Архитектура такого типа позволяла решить проблему забывания, с которой сталкивались рекурентные сети старых архитектур. Проблема тех сетей заключалась в том, что они хорошо улавливали краткосрочные колебания и зависимости, но были нечувствительны к глобальным трендам и примеры, обработанные несколько десятков циклов назад, забывались и никак не влияли на ответы.

В LSTM-сетях внутренние нейроны сопровождаются сложной системой вентилей (они же гейты, gates), а также векотором, который называется состоянием ячейки (cell state), который и представляет собой некий вид долгосрочной памяти. Гейты же определяют, какая информация попадет в состоянием ячейки, какая сотрется из него, и какая повлияет на результат, который выдаст РНС на данном шаге. Чтобы реализовать некоторую модель памяти потребуется три главных гейта:

1. *Гейт остаточной памяти* (remember gate), он же *гейт забывания* (forget gate) - позволяет модели адекватно реагировать на смену контекста, оставляя при этом категории, сохранаяющиеся вне конктекста. На пример анализа текста, сменой контекста может быть новое предложение или абзац. Однако для того, чтобы связать местоимения из нового предложения с подлежащим предыдущего предложения (вспомним шаг разрешения кореференций из NLP), подлежащее нужно оставить в состоянии ячейки.
1. *Гейт добаления* (save gate) - определяет, какая информация из вновь поступившей имеет значение и, следовательно, должна быть помещена в вектор состояния.
1. *Гейт фокусировки* (focus gate), он же *гейт внимания* (attention gate) - модель должна иметь возможность определить, какие элементы долговременной памяти могут пригодиться в самое ближайшее время. Это те самые переменные контекста. Например, если в предложении о Джоне были упомянуты дети, то возраст будет относится к детям, а не к Джону.

### Строение LSTM

Рассмотрим внутреннее строение LSTM-ячейки в самом общем виде:

<center><img src='img/LSTM.png' width=700></center>

И разберём по частям его составляющие. В первую очередь, обратим на вектор, который проходит в верхней части каждого блока, соединяя их между собой:

<center><img src='img/LSTM1.png' width=900></center>

Это и есть *вектор состояния ячейки*. Тот самый элемент, который позволяет LSTM-сетям "запоминать" историю обучения. Если посмотреть на общую схему, видно, что этот вектор изменяется дважды. Первое изменение происходит при прохождении *гейта забывания*:

<center><img src='img/LSTM2.png' width=500></center>

В *гейте забывания* спрятам полносвязный слой, с сигмоидой в качестве активационной функции. Розовый круг на схеме - это операция поэлементного умножения. Сигмоида возвращает числа от нуля до единицы. При поэлементном умножении ноль можно интерпретировать как "забыть этот участок памяти", единица - "не изменять".

Следом идёт *гейт добавления*. 

<center><img src='img/LSTM3.png' width=500></center>

Слой, активируемый гиперболическим тангенсом, формирует вектор значений-кандидатов для нового состояния, а слой с сигмоидой выполняет примерно ту же роль, что и на гейте забывания - определяет, какие веса нужно обновлять. После поэлементного умножения, то есть, выбора весов, которые нужно обновлять, результирующий вектор поэлементно складывается с текущим вектором состояния.

Наконец, нужно решить, какую информацию мы хотим получать на выходе. Этим займётся *гейт фокусировки*.

<center><img src='img/LSTM4.png' width=500></center>

Выходные данные будут основаны на нашем состоянии ячейки, к ним будут применены некоторые фильтры. Сначала входной вектор пройдёт полносвязный слой с сигмоидой, который решит, какуая информация пойдёт дальше, а какая отбросится. Затем значения состояния ячейки проходят через tanh-слой, чтобы получить на выходе значения из диапазона от -1 до 1, и перемножаются с выходными значениями сигмоидального слоя, что позволяет выводить только требуемую информацию.

## Gated Recurrent Units (GRU)

Ещё одна популярная, помимо LSTM, архитектура рекуррентных сейтей - это управляемые рекуррентные нейроны (Gated recurrent units, GRU), впервые описанные в 2014 году. В GRU гейты делятся на *гейт обновления* (update gate) и *гейт сброса* (reset gate). 

<center><img src='img/GRU.png' width=500></center>

Кроме того, состояние ячейки объединяется со скрытым состоянием, есть и другие небольшие изменения. Построенная в результате модель легче (в плане количества параметров), чем стандартная LSTM, и популярность ее неуклонно возрастает.

Рассмотрим процесс прохождения ячейки GRU по шагам. Вычисления начинаются с прохождения *гейта обновления*. Как и в случае с LSTM, такой гейт представляет собой полносвязный слой с сигмоидой:

<center><img src='img/GRU1.png' width=500></center>

Входной вектор $x_t$ и выходной вектор предыдущего шага $h_{t-1}$ конкатенируются в $[x_t, h_{t-1}]$ и проходят через полносвязный слой с сигмоидой, для того, чтобы определить, какая информация должна пройти далее, а какую можно забыть. Результатом этого гейта будет вектор $z_t$. 

Далее такой же вектор $[x_t, h_{t-1}]$ проходит через другой полносвязный слой с сигмоидой для того, чтобы определить, какую информацию из памяти можно сбросить. Эта конструкция является *гейтом сброса*.

<center><img src='img/GRU2.png' width=500></center>

Результат прохождения гейта сброса - вектор $r_t$. Затем идёт пересчёт нового состояния памяти.

<center><img src='img/GRU3.png' width=500></center>

Вычисляется он конкатенацией $x_t$ c вектором $r_t * h_{t-1}$, то есть, с вектором состояния памяти, из которого сброшены те элементы, которые пора забыть. После чего результирующий вектор проходит через слой с гиперболическим тангенсом в качестве активационной функции. Результатом будет вектор $h^{\prime}_t$.

На последнем шаге вычисляется новый выходной вектор $h_t$. 

<center><img src='img/GRU4.png' width=500></center>

По шагам:
1. Сначала поэлементно перемножаются $z_t$ и $h_t$, то есть, происходит изменение вектора состояния результатом гейта обновления.
2. Поэлементно перемножаются $h^{\prime}_t$ и вектор $1 - z_t$. То есть, в сброшенные ячейки записывается то, что стало результатом пересчёта вектора памяти.
3. Векторы, полученные на двух предыдущих шагах конкатенируют. Это и будет $h_t$.

Более подробный разбор GRU можно найти в этом [видео Эндрю Ына](https://www.youtube.com/watch?v=pYRIOGTPRPU). Всё по шагам, с иллюстрациями и комментариями.